# Testing ETL Process
## Extract pdf and info from intranet site

In [ ]:
# Prepare a function to read the configuration file with the confidential data...
# ...because the website is an private website on an Intranet, the production file is config.yaml.
# In GitHub there is only a config.yaml.example with dummy data

import yaml

__config = None

def config():

    global __config

    if not __config:
        with open('./config.yaml', mode='r') as f:
            config = yaml.safe_load(f)

    return config

In [ ]:
# Get the url on the config file

url = config()['websites']['alta_patronal']['url']
user = config()['websites']['alta_patronal']['user']
pwd = config()['websites']['alta_patronal']['pwd']
url_stats = config()['websites']['alta_patronal']['url_stats']
date1 = config()['websites']['alta_patronal']['date1']
date2 = config()['websites']['alta_patronal']['date2']
download_dir = config()['websites']['alta_patronal']['download_dir']
path_to_chromedriver = config()['websites']['path_to_chromedriver']


In [ ]:
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.support import expected_conditions as EC
#from selenium.common.exceptions import TimeoutException

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

import os

options = webdriver.ChromeOptions()

options.add_experimental_option('prefs', {
  'download.prompt_for_download': False,
  'download.default_directory': str(os.path.dirname(os.path.abspath(os.getcwd()))) + download_dir,
  'download.directory_upgrade': True,
  'safebrowsing.enabled': True
})

driver = webdriver.Chrome(executable_path=path_to_chromedriver, options=options)

driver.get(url)

In [ ]:
# Find the fields user and pwd
driver.find_element_by_xpath('//input[@id="cveUsuario"]').get_attribute('id')

In [ ]:
driver.find_element_by_xpath('//input[@id="contrasenia"]').get_attribute('id')

In [ ]:
# Send the credentials
driver.find_element_by_xpath('//input[@id="cveUsuario"]').send_keys(user)

In [ ]:
driver.find_element_by_xpath('//input[@id="contrasenia"]').send_keys(pwd)

In [ ]:
# Locate button to log in
driver.find_element_by_xpath('//input[@type="button"]').get_attribute('name')

In [ ]:
# Log in
driver.find_element_by_xpath('//input[@type="button"]').click()

In [ ]:
# Revisar mensaje de error en login
driver.find_element_by_xpath('//font[@color="red"]/div')

In [ ]:
# Read the error message
driver.find_element_by_xpath('//font[@color="red"]/div').text

In [ ]:
driver.get(url_stats)

In [ ]:
# Locate component date
driver.find_element_by_xpath('//input[@name="fechaInicio"]').get_attribute('id')

In [ ]:
driver.find_element_by_xpath('//input[@name="fechaFin"]').get_attribute('id')

In [ ]:
# Send the first date
driver.find_element_by_xpath('//input[@name="fechaInicio"]').send_keys(date1)

In [ ]:
# Send the second date
driver.find_element_by_xpath('//input[@name="fechaFin"]').send_keys(date2)

In [ ]:
# Now, the next element is a select menu, so we need to import Select from Selenium
from selenium.webdriver.support.select import Select


In [ ]:
# Locate the element
driver.find_element_by_xpath('//select[@id="slcTipoReporte"]').get_attribute('name')

In [ ]:
# Inspect the select and print the options
# First we get the name of the select menu

test = Select(driver.find_element_by_xpath('//select[@id="slcTipoReporte"]'))

#print(test.options[0].text)
#print(test.options[4].text)

In [ ]:
# Select the correct option 
test.select_by_value('4')

In [ ]:
# Locate the button to get the report
driver.find_element_by_xpath('//input[@type="button"]')

In [ ]:
# Click the button
driver.find_element_by_xpath('//input[@type="button"]').click()

In [ ]:
# Close Driver
driver.close()

## Successful file download

In [2]:
# Install module
!pip install PyPDF2

In [3]:
import PyPDF2

# Read the PDF
pdfFileObj = open('../downloads/Reporte2020.pdf', 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
doc_info = PyPDF2.PdfFileReader(pdfFileObj).getDocumentInfo()

# Show numPages
pdfReader.numPages

ModuleNotFoundError: No module named 'PyPDF2'

In [ ]:
doc_info

In [ ]:
pageObj = pdfReader.getPage(0)

In [ ]:
pageObj.extractText()

In [5]:
!pip install tabula-py

In [6]:
# Extract using tabula
import tabula

# Read pdf into a list of DataFrame
#dfs = tabula.read_pdf("test.pdf", pages='all')

# Read remote pdf into a list of DataFrame
#dfs2 = tabula.read_pdf("https://github.com/tabulapdf/tabula-java/raw/master/src/test/resources/technology/tabula/arabic.pdf")

# convert PDF into CSV
tabula.convert_into("../downloads/Reporte2020.pdf", "output_tabula_2020.csv", output_format="csv", pages='all')

# convert all PDFs in a directory
#tabula.convert_into_by_batch("input_directory", output_format='csv', pages='all')

ModuleNotFoundError: No module named 'tabula'

In [ ]:
# Try using tabula jar
!java -jar ../../tabula/tabula-1.0.4-jar-with-dependencies.jar ../downloads/Reporte2020.pdf --format CSV --pages all --outfile Reporte_AP_Reingenieria2020.csv

   ## Check and Transform Data

In [8]:
import pandas as pd

data_ap = pd.read_csv('../downloads/Consulta completa para Tableau2.csv', encoding='Windows 1252')
type(data_ap)
data_ap


,Municipio IMSS,n_CveCIZ,t_SiglasEntSINDO,n_CveEntIMSS,n_CveDelegacion,n_CveSubdelegacion,Fecha Inicio,DíadelaSemana,HoraTrámite,Hora Inicio,Min trámite
0,Z33,2,SIN,25,26,1,2/9/2013 00:00:00,Lunes,13,30/12/1899 13:18:00,46.53
1,Y46,3,PUE,21,22,22,3/9/2013 00:00:00,Martes,11,30/12/1899 11:20:00,18.91
2,A62,1,CHIS,7,7,1,3/9/2013 00:00:00,Martes,11,30/12/1899 11:38:00,2.66
3,Y54,1,DF,9,39,16,3/9/2013 00:00:00,Martes,11,30/12/1899 11:40:00,2.26
4,E64,2,SON,26,27,1,4/9/2013 00:00:00,Miércoles,11,30/12/1899 11:02:00,5.05
...,...,...,...,...,...,...,...,...,...,...,...
1187569,M47,2,BCS,3,3,1,31/12/2019 00:00:00,Martes,11,30/12/1899 11:27:00,1.65
1187570,A48,3,COL,6,6,3,31/12/2019 00:00:00,Martes,11,30/12/1899 11:23:00,7.01
1187571,Y45,3,AGS,1,1,19,31/12/2019 00:00:00,Martes,11,30/12/1899 11:57:00,3.65
1187572,Y37,2,NL,19,20,32,31/12/2019 00:00:00,Martes,12,30/12/1899 12:25:00,2.46


In [9]:
data_ap.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187574 entries, 0 to 1187573
Data columns (total 11 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   Municipio IMSS      1187574 non-null  object 
 1   n_CveCIZ            1187574 non-null  int64  
 2   t_SiglasEntSINDO    1187574 non-null  object 
 3   n_CveEntIMSS        1187574 non-null  int64  
 4   n_CveDelegacion     1187574 non-null  int64  
 5   n_CveSubdelegacion  1187574 non-null  int64  
 6   Fecha Inicio        1187574 non-null  object 
 7   DíadelaSemana       1187574 non-null  object 
 8   HoraTrámite         1187574 non-null  int64  
 9   Hora Inicio         1187574 non-null  object 
 10  Min trámite         1187574 non-null  float64
dtypes: float64(1), int64(5), object(5)
memory usage: 77.0+ MB


In [10]:
data_ap.sample(5)

,Municipio IMSS,n_CveCIZ,t_SiglasEntSINDO,n_CveEntIMSS,n_CveDelegacion,n_CveSubdelegacion,Fecha Inicio,DíadelaSemana,HoraTrámite,Hora Inicio,Min trámite
916003,Y64,1,DF,9,40,1,3/3/2017 00:00:00,Viernes,14,30/12/1899 14:36:00,4.01
536999,E23,1,QRO,22,23,1,27/8/2010 00:00:00,Viernes,14,30/12/1899 14:31:00,13.13
489995,B52,3,GTO,11,11,14,28/1/2011 00:00:00,Viernes,10,30/12/1899 10:29:00,5.96
450275,D38,2,NL,19,20,8,27/4/2011 00:00:00,Miércoles,10,30/12/1899 10:34:00,3.88
604315,Z33,2,SIN,25,26,1,26/8/2014 00:00:00,Martes,12,30/12/1899 12:42:00,5.81


In [13]:
data_ap['Min trámite'].value_counts()

2.43      1974
2.56      1918
2.20      1916
2.93      1914
2.00      1903
          ... 
113.93       1
186.90       1
137.61       1
104.05       1
85.65        1
Name: Min trámite, Length: 5965, dtype: int64

In [11]:
data_ap['n_CveEntIMSS'].value_counts()

14    120946
9     119580
19     83522
15     77818
30     67289
11     59508
25     44031
2      42915
26     40697
21     40172
5      37395
16     36966
8      36026
22     35240
28     31334
20     25994
24     25360
23     23818
31     23508
1      22656
32     22580
12     21945
7      19383
13     19368
3      16793
17     16686
18     16253
27     15540
10     14631
6      13739
29      8342
4       7539
Name: n_CveEntIMSS, dtype: int64

### Missing data?

In [17]:
missing_time_mask = data_ap['Min trámite'].isna()
missing_time_mask

0          False
1          False
2          False
3          False
4          False
           ...  
1187569    False
1187570    False
1187571    False
1187572    False
1187573    False
Name: Min trámite, Length: 1187574, dtype: bool

In [20]:
missing_time_mask.value_counts()

False    1187574
Name: Min trámite, dtype: int64

In [21]:
missing_ent_mask = data_ap['n_CveEntIMSS'].isna()
missing_ent_mask.value_counts()

False    1187574
Name: n_CveEntIMSS, dtype: int64

In [25]:
# Change the notation, and fix format
pd.options.display.float_format = '{:,.2f}'.format

In [26]:
data_ap.describe()

,n_CveCIZ,n_CveEntIMSS,n_CveDelegacion,n_CveSubdelegacion,HoraTrámite,Min trámite
count,"1,187,574.00","1,187,574.00","1,187,574.00","1,187,574.00","1,187,574.00","1,187,574.00"
mean,2.15,16.30,19.93,15.90,11.16,10.20
std,0.80,8.31,10.70,18.85,2.09,10.54
min,1.00,1.00,1.00,1.00,5.00,-5.30
25%,1.00,9.00,13.00,1.00,9.00,3.40
50%,2.00,15.00,20.00,7.00,11.00,6.90
75%,3.00,23.00,27.00,25.00,13.00,13.31
max,3.00,32.00,40.00,80.00,23.00,342.25


In [27]:
data_ap.describe(include='all')

,Municipio IMSS,n_CveCIZ,t_SiglasEntSINDO,n_CveEntIMSS,n_CveDelegacion,n_CveSubdelegacion,Fecha Inicio,DíadelaSemana,HoraTrámite,Hora Inicio,Min trámite
count,1187574,"1,187,574.00",1187574,"1,187,574.00","1,187,574.00","1,187,574.00",1187574,1187574,"1,187,574.00",1187574,"1,187,574.00"
unique,2347,nan,32,nan,nan,nan,2503,7,nan,895,nan
top,Z29,nan,JAL,nan,nan,nan,18/5/2012 00:00:00,Miércoles,nan,30/12/1899 10:32:00,nan
freq,29046,nan,120946,nan,nan,nan,1169,253311,nan,3494,nan
mean,NaN,2.15,NaN,16.30,19.93,15.90,NaN,NaN,11.16,NaN,10.20
std,NaN,0.80,NaN,8.31,10.70,18.85,NaN,NaN,2.09,NaN,10.54
min,NaN,1.00,NaN,1.00,1.00,1.00,NaN,NaN,5.00,NaN,-5.30
25%,NaN,1.00,NaN,9.00,13.00,1.00,NaN,NaN,9.00,NaN,3.40
50%,NaN,2.00,NaN,15.00,20.00,7.00,NaN,NaN,11.00,NaN,6.90
75%,NaN,3.00,NaN,23.00,27.00,25.00,NaN,NaN,13.00,NaN,13.31


In [28]:
data_ap.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187574 entries, 0 to 1187573
Data columns (total 11 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   Municipio IMSS      1187574 non-null  object 
 1   n_CveCIZ            1187574 non-null  int64  
 2   t_SiglasEntSINDO    1187574 non-null  object 
 3   n_CveEntIMSS        1187574 non-null  int64  
 4   n_CveDelegacion     1187574 non-null  int64  
 5   n_CveSubdelegacion  1187574 non-null  int64  
 6   Fecha Inicio        1187574 non-null  object 
 7   DíadelaSemana       1187574 non-null  object 
 8   HoraTrámite         1187574 non-null  int64  
 9   Hora Inicio         1187574 non-null  object 
 10  Min trámite         1187574 non-null  float64
dtypes: float64(1), int64(5), object(5)
memory usage: 77.0+ MB


In [29]:
data_ap.dtypes

Municipio IMSS         object
n_CveCIZ                int64
t_SiglasEntSINDO       object
n_CveEntIMSS            int64
n_CveDelegacion         int64
n_CveSubdelegacion      int64
Fecha Inicio           object
DíadelaSemana          object
HoraTrámite             int64
Hora Inicio            object
Min trámite           float64
dtype: object

In [30]:
data_ap.convert_dtypes().dtypes

Municipio IMSS         string
n_CveCIZ                Int64
t_SiglasEntSINDO       string
n_CveEntIMSS            Int64
n_CveDelegacion         Int64
n_CveSubdelegacion      Int64
Fecha Inicio           string
DíadelaSemana          string
HoraTrámite             Int64
Hora Inicio            string
Min trámite           float64
dtype: object

In [31]:
data_ap.sample(10)

,Municipio IMSS,n_CveCIZ,t_SiglasEntSINDO,n_CveEntIMSS,n_CveDelegacion,n_CveSubdelegacion,Fecha Inicio,DíadelaSemana,HoraTrámite,Hora Inicio,Min trámite
511482,Y66,1,DF,9,40,11,7/10/2010 00:00:00,Jueves,14,30/12/1899 14:03:00,11.01
654332,Z06,3,GTO,11,11,17,14/1/2015 00:00:00,Miércoles,8,30/12/1899 08:41:00,5.58
693701,J91,3,OAX,20,21,53,30/4/2015 00:00:00,Jueves,10,30/12/1899 10:54:00,2.91
265712,E68,2,SON,26,27,10,3/4/2012 00:00:00,Martes,15,30/12/1899 15:52:00,3.83
687154,Z29,3,JAL,14,14,38,24/4/2015 00:00:00,Viernes,11,30/12/1899 11:29:00,7.36
892123,H01,2,ZAC,32,34,9,16/12/2016 00:00:00,Viernes,9,30/12/1899 09:11:00,5.88
1137599,A34,2,COAH,5,5,3,6/6/2019 00:00:00,Jueves,8,30/12/1899 08:19:00,2.08
106824,C67,1,MEX,15,16,1,11/3/2013 00:00:00,Lunes,9,30/12/1899 09:10:00,19.33
351040,D45,2,NL,19,20,33,19/1/2012 00:00:00,Jueves,9,30/12/1899 09:16:00,6.15
650505,A65,1,CHIS,7,7,2,16/1/2015 00:00:00,Viernes,8,30/12/1899 08:05:00,35.41


In [32]:
data_ap.describe(include='all')

,Municipio IMSS,n_CveCIZ,t_SiglasEntSINDO,n_CveEntIMSS,n_CveDelegacion,n_CveSubdelegacion,Fecha Inicio,DíadelaSemana,HoraTrámite,Hora Inicio,Min trámite
count,1187574,"1,187,574.00",1187574,"1,187,574.00","1,187,574.00","1,187,574.00",1187574,1187574,"1,187,574.00",1187574,"1,187,574.00"
unique,2347,nan,32,nan,nan,nan,2503,7,nan,895,nan
top,Z29,nan,JAL,nan,nan,nan,18/5/2012 00:00:00,Miércoles,nan,30/12/1899 10:32:00,nan
freq,29046,nan,120946,nan,nan,nan,1169,253311,nan,3494,nan
mean,NaN,2.15,NaN,16.30,19.93,15.90,NaN,NaN,11.16,NaN,10.20
std,NaN,0.80,NaN,8.31,10.70,18.85,NaN,NaN,2.09,NaN,10.54
min,NaN,1.00,NaN,1.00,1.00,1.00,NaN,NaN,5.00,NaN,-5.30
25%,NaN,1.00,NaN,9.00,13.00,1.00,NaN,NaN,9.00,NaN,3.40
50%,NaN,2.00,NaN,15.00,20.00,7.00,NaN,NaN,11.00,NaN,6.90
75%,NaN,3.00,NaN,23.00,27.00,25.00,NaN,NaN,13.00,NaN,13.31


In [33]:
data_ap.nunique()

Municipio IMSS        2347
n_CveCIZ                 3
t_SiglasEntSINDO        32
n_CveEntIMSS            32
n_CveDelegacion         35
n_CveSubdelegacion      41
Fecha Inicio          2503
DíadelaSemana            7
HoraTrámite             19
Hora Inicio            895
Min trámite           5965
dtype: int64

In [34]:
data_ap['Fecha Inicio']

0            2/9/2013 00:00:00
1            3/9/2013 00:00:00
2            3/9/2013 00:00:00
3            3/9/2013 00:00:00
4            4/9/2013 00:00:00
                  ...         
1187569    31/12/2019 00:00:00
1187570    31/12/2019 00:00:00
1187571    31/12/2019 00:00:00
1187572    31/12/2019 00:00:00
1187573    31/12/2019 00:00:00
Name: Fecha Inicio, Length: 1187574, dtype: object

In [38]:
# Transform the values to datetime
# 'coerce': Allow to convert no datetime values to non-numeric value
pd.to_datetime(
    data_ap['Fecha Inicio'],
    errors = 'coerce',
    format = '%d/%m/%Y %H:%M:%S'
)

0         2013-09-02
1         2013-09-03
2         2013-09-03
3         2013-09-03
4         2013-09-04
             ...    
1187569   2019-12-31
1187570   2019-12-31
1187571   2019-12-31
1187572   2019-12-31
1187573   2019-12-31
Name: Fecha Inicio, Length: 1187574, dtype: datetime64[ns]

In [39]:
data_ap['Fecha Inicio'] = pd.to_datetime(
    data_ap['Fecha Inicio'],
    errors = 'coerce',
    format = '%d/%m/%Y %H:%M:%S'
)

In [40]:
data_ap.dtypes

Municipio IMSS                object
n_CveCIZ                       int64
t_SiglasEntSINDO              object
n_CveEntIMSS                   int64
n_CveDelegacion                int64
n_CveSubdelegacion             int64
Fecha Inicio          datetime64[ns]
DíadelaSemana                 object
HoraTrámite                    int64
Hora Inicio                   object
Min trámite                  float64
dtype: object

In [41]:
data_ap.convert_dtypes().dtypes

Municipio IMSS                string
n_CveCIZ                       Int64
t_SiglasEntSINDO              string
n_CveEntIMSS                   Int64
n_CveDelegacion                Int64
n_CveSubdelegacion             Int64
Fecha Inicio          datetime64[ns]
DíadelaSemana                 string
HoraTrámite                    Int64
Hora Inicio                   string
Min trámite                  float64
dtype: object

In [42]:
data_ap.dtypes

Municipio IMSS                object
n_CveCIZ                       int64
t_SiglasEntSINDO              object
n_CveEntIMSS                   int64
n_CveDelegacion                int64
n_CveSubdelegacion             int64
Fecha Inicio          datetime64[ns]
DíadelaSemana                 object
HoraTrámite                    int64
Hora Inicio                   object
Min trámite                  float64
dtype: object

In [43]:
data_ap.rename(columns={'Min trámite':'MinTramite'}, inplace=True)

In [45]:
data_ap.rename(columns={'n_CveEntIMSS':'nCveEntidad'}, inplace=True)

In [47]:
data_ap.rename(columns={'t_SiglasEntSINDO':'abrEntidad'}, inplace=True)

In [48]:
data_ap.sample(10)

,Municipio IMSS,n_CveCIZ,abrEntidad,nCveEntidad,n_CveDelegacion,n_CveSubdelegacion,Fecha Inicio,DíadelaSemana,HoraTrámite,Hora Inicio,MinTramite
195000,F34,3,VER,30,32,45,2012-10-02,Martes,9,30/12/1899 09:03:00,30.26
54746,Y68,1,DF,9,40,54,2013-05-15,Miércoles,11,30/12/1899 11:53:00,18.63
740778,D16,1,MOR,17,18,1,2015-09-07,Lunes,11,30/12/1899 11:47:00,7.53
1106994,A83,2,CHIH,8,8,10,2019-02-28,Jueves,16,30/12/1899 16:05:00,23.98
848317,Z29,3,JAL,14,14,38,2016-07-12,Martes,9,30/12/1899 09:24:00,15.63
575445,Y50,1,DF,9,39,11,2010-05-18,Martes,9,30/12/1899 09:44:00,10.55
421677,E23,1,QRO,22,23,1,2011-06-09,Jueves,13,30/12/1899 13:44:00,4.15
482145,E26,1,QRO,22,23,1,2011-01-10,Lunes,10,30/12/1899 10:43:00,2.88
832797,Z06,3,GTO,11,11,17,2016-06-01,Miércoles,15,30/12/1899 15:10:00,11.76
1182440,Z38,3,YUC,31,33,1,2019-11-28,Jueves,8,30/12/1899 08:50:00,9.50


In [49]:
data_ap.to_csv('Data_AP.csv')

In [60]:
# Add prefix "0"
'0' + data_ap['nCveEntidad'].astype(str)

0          025
1          021
2           07
3           09
4          026
          ... 
1187569     03
1187570     06
1187571     01
1187572    019
1187573    015
Name: nCveEntidad, Length: 1187574, dtype: object

In [63]:
# Take the last two chars
('0' + data_ap['nCveEntidad'].astype(str)).str[-2:]

0          25
1          21
2          07
3          09
4          26
           ..
1187569    03
1187570    06
1187571    01
1187572    19
1187573    15
Name: nCveEntidad, Length: 1187574, dtype: object

In [64]:
(('0' + data_ap['nCveEntidad'].astype(str)).str[-2:]).astype(str)

0          25
1          21
2          07
3          09
4          26
           ..
1187569    03
1187570    06
1187571    01
1187572    19
1187573    15
Name: nCveEntidad, Length: 1187574, dtype: object

In [65]:
data_ap['nCveEntidad'] = (('0' + data_ap['nCveEntidad'].astype(str)).str[-2:]).astype(str)

In [66]:
data_ap.sample(10)

,Municipio IMSS,n_CveCIZ,abrEntidad,nCveEntidad,n_CveDelegacion,n_CveSubdelegacion,Fecha Inicio,DíadelaSemana,HoraTrámite,Hora Inicio,MinTramite
975860,Z39,1,DF,09,40,54,2017-09-19,Martes,11,30/12/1899 11:54:00,9.76
863943,Y58,1,DF,09,39,57,2016-09-08,Jueves,12,30/12/1899 12:45:00,14.16
575066,V74,3,OAX,20,21,3,2010-05-12,Miércoles,12,30/12/1899 12:05:00,5.20
155924,E64,2,SON,26,27,1,2014-02-18,Martes,13,30/12/1899 13:12:00,10.06
448043,Z29,3,JAL,14,14,38,2011-04-07,Jueves,9,30/12/1899 09:54:00,12.83
630811,E97,2,TAM,28,29,18,2014-11-06,Jueves,9,30/12/1899 09:18:00,1.40
446534,F33,3,VER,30,31,2,2011-04-04,Lunes,13,30/12/1899 13:29:00,5.55
779643,A01,3,AGS,01,1,1,2016-01-18,Lunes,11,30/12/1899 11:45:00,4.76
376470,H64,3,JAL,14,14,12,2011-10-06,Jueves,12,30/12/1899 12:37:00,12.33
221081,D35,3,NAY,18,19,1,2012-08-09,Jueves,10,30/12/1899 10:31:00,1.76


In [67]:
data_ap.to_csv('Data_AP2.csv')

In [68]:
data_ap.dtypes

Municipio IMSS                object
n_CveCIZ                       int64
abrEntidad                    object
nCveEntidad                   object
n_CveDelegacion                int64
n_CveSubdelegacion             int64
Fecha Inicio          datetime64[ns]
DíadelaSemana                 object
HoraTrámite                    int64
Hora Inicio                   object
MinTramite                   float64
dtype: object

In [69]:
data_ap2 = data_ap.convert_dtypes()

In [70]:
data_ap2.dtypes

Municipio IMSS                string
n_CveCIZ                       Int64
abrEntidad                    string
nCveEntidad                   string
n_CveDelegacion                Int64
n_CveSubdelegacion             Int64
Fecha Inicio          datetime64[ns]
DíadelaSemana                 string
HoraTrámite                    Int64
Hora Inicio                   string
MinTramite                   float64
dtype: object

In [71]:
data_ap2.to_csv('Data_AP3.csv')

In [73]:
data_ap_sample = data_ap2.sample(500000)

In [74]:
data_ap_sample.to_csv('Data_AP_Sample.csv')